# Capital Asset Pricing Model

$$
ER_i = R_f + \beta_i(ER_m - R_f)
$$

- $ER_i$ = Expected Return of Asset
- $ER_m$ = Expected Return of Market
- $R_f$ = Risk-free rate
- $\beta_i$ = Beta of the Investment (This is a relative risk measurement with respect to the market/ slope or average on daily/monthly depending how we want to tweak our model)
- $(ER_m - R_f)$ = Market Risk Premium

In [4]:
!pip install statsmodels
import re
from io import StringIO
from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np
import pandas_datareader as pdr
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 351.6 kB/s eta 0:00:0000:0100:02


In [6]:
# Loading in our data
def get_historical_Data(tickers):
    """
    This function returns a pd dataframe with all of the adjusted closing information
    """
    data = pd.DataFrame()
    names = list()
    for i in tickers:
        data = pd.concat([data,pdr.get_data_yahoo(symbols=i, start=datetime(2014, 10, 3), end=datetime(2024, 10, 3)).iloc[:,5]], axis = 1)
        names.append(i)
    data.columns = names
    return data

In [8]:
# Reading in our 10 year Treasury Constant Maturity Rate.
# https://fred.stlouisfed.org/series/DGS10
#DGS/10 = 10 year treasury constant maturity rate
T_rate = pd.read_csv('/Users/vimalvatsa/portfolio-optimization/Portfolio_optimization_techniques/DGS10.csv')

In [9]:
T_rate['DATE'] = pd.to_datetime(T_rate['DATE'], format = "%Y-%m-%d").dt.strftime('%m-%d-%Y')

In [10]:
T_rate

,DATE,DGS10
0,10-01-2019,1.65
1,10-02-2019,1.60
2,10-03-2019,1.54
3,10-04-2019,1.52
4,10-07-2019,1.56
...,...,...
1301,09-25-2024,3.79
1302,09-26-2024,3.79
1303,09-27-2024,3.75
1304,09-30-2024,3.81


In [11]:
#since we don't have a sparse dataset to work with we assume that the data is continuous and we can use the data as is
count = 0
for i in T_rate['DGS10']:
    if i =='.':
        T_rate['DGS10'][count] = T_rate['DGS10'][count-1]
    count+=1

/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/3943343393.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  T_rate['DGS10'][count] = T_rate['DGS10'][count-1]
/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_5023

In [12]:
T_rate

,DATE,DGS10
0,10-01-2019,1.65
1,10-02-2019,1.60
2,10-03-2019,1.54
3,10-04-2019,1.52
4,10-07-2019,1.56
...,...,...
1301,09-25-2024,3.79
1302,09-26-2024,3.79
1303,09-27-2024,3.75
1304,09-30-2024,3.81


In [14]:
import yfinance as yf  # Add this import

def get_historical_Data(tickers):
    """
    This function returns a pd dataframe with all of the adjusted closing information
    """
    data = pd.DataFrame()
    names = list()
    for i in tickers:
        # Use yfinance to download data
        stock_data = yf.download(i, start="2014-10-03", end="2024-10-03")
        data = pd.concat([data, stock_data['Adj Close']], axis=1)
        names.append(i)
    data.columns = names
    return data

In [65]:
#    The ticker names of the companies that we will be looking at. (And the S&P500)
#WFC - Wells Fargo & Company, ^GSPC - S&P 500 ticker symbol
ticks = ["WFC", '^GSPC']
d = get_historical_Data(ticks)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [66]:
d = d.reset_index()
for i in range(d.shape[0]):
    mo = ''
    da = ''
    if d['index'][i].month < 10:
        mo = '0' + str(d['index'][i].month)
    else:
        mo = str(d['index'][i].month)
    if d['index'][i].day < 10:
        da = '0' + str(d['index'][i].day)
    else:
        da = str(d['index'][i].day)
    d['index'][i] = mo + '-' + da + '-' + str(d['index'][i].year) 
# Changing the index name to date
d = d.rename(columns = {"index": "DATE"})

/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/2693000346.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  d['index'][i] = mo + '-' + da + '-' + str(d['index'][i].year)
/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/i

In [67]:
d

,DATE,WFC,^GSPC
0,2014-10-03,39.048973,1967.900024
1,2014-10-06,38.996517,1964.819946
2,2014-10-07,38.284489,1935.099976
3,2014-10-08,39.086456,1968.890015
4,2014-10-09,38.314461,1928.209961
...,...,...,...
2511,2024-09-26,56.389999,5745.370117
2512,2024-09-27,55.900002,5738.169922
2513,2024-09-30,56.490002,5762.479980
2514,2024-10-01,55.389999,5708.750000


In [69]:
# Convert 'DATE' in T_rate to datetime
T_rate['DATE'] = pd.to_datetime(T_rate['DATE'], errors='coerce')

# Convert 'DATE' in d to datetime (if not already done)
d['DATE'] = pd.to_datetime(d['DATE'], errors='coerce')

In [70]:
# Merge dataframes
data = pd.merge(left = T_rate, right = d, left_on = 'DATE', right_on = 'DATE')
data = data.rename(columns = {"DGS10": "10_Year_Rate"})

In [72]:
data
#we have the well's fargo stock price (closing price) and the 10 year treasury rate data for the same time period 

,DATE,10_Year_Rate,WFC,^GSPC
0,2019-10-01,1.65,42.785461,2940.250000
1,2019-10-02,1.60,42.270931,2887.610107
2,2019-10-03,1.54,42.279644,2910.629883
3,2019-10-04,1.52,42.916279,2952.010010
4,2019-10-07,1.56,42.567451,2938.790039
...,...,...,...,...
1254,2024-09-25,3.79,53.610001,5722.259766
1255,2024-09-26,3.79,56.389999,5745.370117
1256,2024-09-27,3.75,55.900002,5738.169922
1257,2024-09-30,3.81,56.490002,5762.479980


In [82]:
# Because of the CAPM formula, we need to calculate the Percent changes of our given assets.
data['WFC_Daily_Returns'] = data['WFC'].pct_change()
data['SP500_Daily_Returns'] = data['^GSPC'].pct_change()

In [83]:
data

,DATE,10_Year_Rate,WFC,^GSPC,DPZ_Daily_Returns,SP500_Daily_Returns,WFC_Daily_Returns
0,2019-10-01,1.65,42.785461,2940.250000,NaN,NaN,NaN
1,2019-10-02,1.60,42.270931,2887.610107,-0.012026,-0.017903,-0.012026
2,2019-10-03,1.54,42.279644,2910.629883,0.000206,0.007972,0.000206
3,2019-10-04,1.52,42.916279,2952.010010,0.015058,0.014217,0.015058
4,2019-10-07,1.56,42.567451,2938.790039,-0.008128,-0.004478,-0.008128
...,...,...,...,...,...,...,...
1254,2024-09-25,3.79,53.610001,5722.259766,-0.010155,-0.001861,-0.010155
1255,2024-09-26,3.79,56.389999,5745.370117,0.051856,0.004039,0.051856
1256,2024-09-27,3.75,55.900002,5738.169922,-0.008689,-0.001253,-0.008689
1257,2024-09-30,3.81,56.490002,5762.479980,0.010555,0.004237,0.010555


In [84]:
# Taking out the first row as it is NaN
data = data[1:]

# Calculate the excess return, which is calculating the expected return for your given company, subtracting from the risk free rate, and then subtracting that from the expected return of the market.
#which includes the 10 year rate relating to the overall return 
data['Excess_Return_WFC'] = data['WFC_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100
data['Excess_Return_SP500'] = data['SP500_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100

/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/4143013043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Excess_Return_WFC'] = data['WFC_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100
/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/4143013043.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Excess_Return_SP500'] = data['SP500_Daily_Returns'] - data['10_Year_Rate'].astype('float') / 100


In [85]:
data

,DATE,10_Year_Rate,WFC,^GSPC,DPZ_Daily_Returns,SP500_Daily_Returns,WFC_Daily_Returns,Excess_Return_WFC,Excess_Return_SP500
1,2019-10-02,1.60,42.270931,2887.610107,-0.012026,-0.017903,-0.012026,-0.028026,-0.033903
2,2019-10-03,1.54,42.279644,2910.629883,0.000206,0.007972,0.000206,-0.015194,-0.007428
3,2019-10-04,1.52,42.916279,2952.010010,0.015058,0.014217,0.015058,-0.000142,-0.000983
4,2019-10-07,1.56,42.567451,2938.790039,-0.008128,-0.004478,-0.008128,-0.023728,-0.020078
5,2019-10-08,1.54,41.704048,2893.060059,-0.020283,-0.015561,-0.020283,-0.035683,-0.030961
...,...,...,...,...,...,...,...,...,...
1254,2024-09-25,3.79,53.610001,5722.259766,-0.010155,-0.001861,-0.010155,-0.048055,-0.039761
1255,2024-09-26,3.79,56.389999,5745.370117,0.051856,0.004039,0.051856,0.013956,-0.033861
1256,2024-09-27,3.75,55.900002,5738.169922,-0.008689,-0.001253,-0.008689,-0.046189,-0.038753
1257,2024-09-30,3.81,56.490002,5762.479980,0.010555,0.004237,0.010555,-0.027545,-0.033863


In [86]:
# Running a regression to calculate Beta
results = smf.ols('Excess_Return_WFC ~ Excess_Return_SP500', data = data).fit()

In [87]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      Excess_Return_WFC   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     1577.
Date:                Fri, 04 Oct 2024   Prob (F-statistic):          4.33e-224
Time:                        02:35:23   Log-Likelihood:                 3242.7
No. Observations:                1258   AIC:                            -6481.
Df Residuals:                    1256   BIC:                            -6471.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0018    

In [88]:
# Beta from the OLS above
Beta = 1.1367
# Once we have Beta, we can calculate the expected return of the company BASED on the market.
average_risk_free_rate = data['10_Year_Rate'].astype('float').mean() /100
# Using the Historical Rate of return for the S&P500 market...
# Including dividends but not accounting for inflation
average_return_SP500 = (1.311 * (1-.0441) * (1.2194) * (1.1193) * (1.0131) * (1.1381))**(1/5) - 1

print('Average Risk Free Rate',average_risk_free_rate)
print('Average Return S&P500',average_return_SP500)
print('Expected Return of DPZ',average_risk_free_rate + Beta * (average_return_SP500 - average_risk_free_rate))

Average Risk Free Rate 0.025651112877583464
Average Return S&P500 0.14548159954598372
Expected Return of DPZ 0.16186242707355403


### Doing the Same thing but for the monthly data.

In [89]:
# Using Monthly data for T Rates
mT_rate = pd.read_csv('/Users/vimalvatsa/portfolio-optimization/Portfolio_optimization_techniques/DGS10.csv')
#https://fred.stlouisfed.org/series/DGS10
count = 0
for i in mT_rate['DGS10']:
    if i =='.':
        mT_rate['DGS10'][count] = mT_rate['DGS10'][count-1]
    count+=1
mT_rate

/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/840412374.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  mT_rate['DGS10'][count] = mT_rate['DGS10'][count-1]


,DATE,DGS10
0,2019-10-01,1.65
1,2019-10-02,1.60
2,2019-10-03,1.54
3,2019-10-04,1.52
4,2019-10-07,1.56
...,...,...
1301,2024-09-25,3.79
1302,2024-09-26,3.79
1303,2024-09-27,3.75
1304,2024-09-30,3.81


In [99]:
import pandas_datareader.data as web
import yfinance as yf

# Monthly Company data
tick = 'WFC'
Mdata = yf.download(tick, start='2021-11-24', end='2024-11-24', interval='1mo')

# Monthly Market data
MMdata = yf.download('^GSPC', start='2021-11-24', end='2024-11-24', interval='1mo')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [100]:
Mdata = Mdata.rename(columns = {"Adj Close": tick})
MMdata = MMdata.rename(columns = {"Adj Close": "SP500"})

In [101]:
Mdata = pd.concat([Mdata[tick], MMdata['SP500'] ], axis = 1)

In [102]:
Mdata

,WFC,SP500
Date,,
2021-12-01,44.541191,4766.180176
2022-01-01,49.944065,4515.549805
2022-02-01,49.544880,4373.939941
2022-03-01,45.188782,4530.410156
2022-04-01,40.684822,4131.930176
2022-05-01,42.680367,4132.149902
2022-06-01,36.725559,3785.379883
2022-07-01,41.132259,4130.290039
2022-08-01,40.982246,3955.000000


In [103]:
#S&P 500 mont to month returns
Mdata = Mdata.reset_index()
for i in range(Mdata.shape[0]):
    mo = ''
    da = ''
    if Mdata['Date'][i].month < 10:
        mo = '0' + str(Mdata['Date'][i].month)
    else:
        mo = str(Mdata['Date'][i].month)
        
    if Mdata['Date'][i].day < 10:
        da = '0' + str(Mdata['Date'][i].day)
    else:
        da = str(Mdata['Date'][i].day)
    Mdata['Date'][i] = str(Mdata['Date'][i].year) + '-' + mo  + '-' +  da
# Changing the index name to date
Mdata = Mdata.rename(columns = {"Date": "DATE"})
Mdata

/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/1122680904.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Mdata['Date'][i] = str(Mdata['Date'][i].year) + '-' + mo  + '-' +  da
/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc00

,DATE,WFC,SP500
0,2021-12-01,44.541191,4766.180176
1,2022-01-01,49.944065,4515.549805
2,2022-02-01,49.544880,4373.939941
3,2022-03-01,45.188782,4530.410156
4,2022-04-01,40.684822,4131.930176
5,2022-05-01,42.680367,4132.149902
6,2022-06-01,36.725559,3785.379883
7,2022-07-01,41.132259,4130.290039
8,2022-08-01,40.982246,3955.000000
9,2022-09-01,37.969582,3585.620117


In [104]:
Mdata['SP500_Daily_Returns'] = Mdata['SP500'].pct_change()
Mdata['{}_Daily_Returns'.format(tick)] = Mdata[tick].pct_change()
Mdata = Mdata[1:]

In [106]:
# Convert 'DATE' in mT_rate to datetime
mT_rate['DATE'] = pd.to_datetime(mT_rate['DATE'], errors='coerce')

# Convert 'DATE' in Mdata to datetime
Mdata['DATE'] = pd.to_datetime(Mdata['DATE'], errors='coerce')

/var/folders/jn/7sdsjp9d0cn_c7zfft_vvyyc0000gn/T/ipykernel_50235/159492018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mdata['DATE'] = pd.to_datetime(Mdata['DATE'], errors='coerce')


In [107]:
# Merging Data Frames
Mdata = pd.merge(left = mT_rate, right = Mdata, left_on = 'DATE', right_on = 'DATE')
Mdata

,DATE,DGS10,WFC,SP500,SP500_Daily_Returns,WFC_Daily_Returns
0,2022-02-01,1.81,49.544880,4373.939941,-0.031360,-0.007993
1,2022-03-01,1.72,45.188782,4530.410156,0.035773,-0.087922
2,2022-04-01,2.39,40.684822,4131.930176,-0.087957,-0.099670
3,2022-06-01,2.94,36.725559,3785.379883,-0.083920,-0.139521
4,2022-07-01,2.88,41.132259,4130.290039,0.091116,0.119990
5,2022-08-01,2.60,40.982246,3955.000000,-0.042440,-0.003647
6,2022-09-01,3.26,37.969582,3585.620117,-0.093396,-0.073511
7,2022-11-01,4.07,45.267063,4080.110107,0.053753,0.042618
8,2022-12-01,3.53,39.230755,3839.500000,-0.058971,-0.133349
9,2023-02-01,3.39,44.437454,3970.149902,-0.026112,-0.002133


In [108]:
Mdata['Excess_Return_{}'.format(tick)] = Mdata['{}_Daily_Returns'.format(tick)] - Mdata['DGS10'].astype('float') / 100
Mdata['Excess_Return_SP500'] = Mdata['SP500_Daily_Returns'] - Mdata['DGS10'].astype('float') / 100
Mdata

,DATE,DGS10,WFC,SP500,SP500_Daily_Returns,WFC_Daily_Returns,Excess_Return_WFC,Excess_Return_SP500
0,2022-02-01,1.81,49.544880,4373.939941,-0.031360,-0.007993,-0.026093,-0.049460
1,2022-03-01,1.72,45.188782,4530.410156,0.035773,-0.087922,-0.105122,0.018573
2,2022-04-01,2.39,40.684822,4131.930176,-0.087957,-0.099670,-0.123570,-0.111857
3,2022-06-01,2.94,36.725559,3785.379883,-0.083920,-0.139521,-0.168921,-0.113320
4,2022-07-01,2.88,41.132259,4130.290039,0.091116,0.119990,0.091190,0.062316
5,2022-08-01,2.60,40.982246,3955.000000,-0.042440,-0.003647,-0.029647,-0.068440
6,2022-09-01,3.26,37.969582,3585.620117,-0.093396,-0.073511,-0.106111,-0.125996
7,2022-11-01,4.07,45.267063,4080.110107,0.053753,0.042618,0.001918,0.013053
8,2022-12-01,3.53,39.230755,3839.500000,-0.058971,-0.133349,-0.168649,-0.094271
9,2023-02-01,3.39,44.437454,3970.149902,-0.026112,-0.002133,-0.036033,-0.060012


In [109]:
# Running a regression to calculate Beta
# Can play around with what risk free rate to use to get your relative beta.
results = smf.ols('Excess_Return_{} ~ Excess_Return_SP500'.format(tick), data = Mdata).fit()

In [110]:
print(results.summary())
#here Intercept is essentially the alpha value and the coefficient of the SP500 is the beta value, its the measure of performance. The higher the intercept the more the performance is and the better it gets
#Hedging is a strategy that is used to offset or limit the loss from another investment. It is a way to protect yourself from losing money.
#Hedge funds try to seek and maximise the alpha value 

                            OLS Regression Results                            
Dep. Variable:      Excess_Return_WFC   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     13.09
Date:                Fri, 04 Oct 2024   Prob (F-statistic):            0.00144
Time:                        02:59:52   Log-Likelihood:                 32.877
No. Observations:                  25   AIC:                            -61.75
Df Residuals:                      23   BIC:                            -59.32
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.0113    

In [111]:
# Beta from the OLS above
Beta = 1.0775
# Once we have Beta, we can calculate the expected return of the company BASED on the market.
average_risk_free_rate = Mdata['DGS10'].astype('float').mean() /100
# Using the Historical Rate of return for the S&P500 market...
# Including dividends but not accounting for inflation
average_return_SP500 = (1.311 * (1-.0441) * (1.2194) * (1.1193) * (1.0131) * (1.1381))**(1/5) - 1

print('Average Risk Free Rate',average_risk_free_rate)
print('Average Return S&P500',average_return_SP500)
print('Expected Return of {}'.format(tick),average_risk_free_rate + Beta * (average_return_SP500 - average_risk_free_rate))

Average Risk Free Rate 0.036051999999999994
Average Return S&P500 0.14548159954598372
Expected Return of WFC 0.15396239351079744
